# Model development and training
In this notebook we will develop a Recurrent Neural Network (RNN) using LSTM nodes, so as to predict the next character, given a sequence of 100 characters.

First of all, we import the numpy and keras modules, important for storing data and defining the model respectively.

In [2]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
SEQ_LENGTH = 100

We define the model now. The model is given an input of 100 character sequences and it outputs the respective probabilities with which a character can succeed the input sequence. The model consists of 3 hidden layers. The first two hidden layers consist of 256 LSTM cells, and the second layer is fully connected to the third layer. The number of neurons in the third layer is same as the number of unique characters in the training set. The neurons in the third layer, use softmax activation so as to convert their outputs into respective probabilities. The loss used is Categorical cross entropy and the optimizer used is Adam.

In [4]:
def buildmodel(VOCABULARY):
    model = Sequential()
    model.add(LSTM(256, input_shape = (SEQ_LENGTH, 1), return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    model.add(Dense(VOCABULARY, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
    return model

Next, we download the training data. The children book "Alice's Adventures in Wonderland" written by Lewis Caroll has been used as the training data in this project. The ebook can be downloaded from [here](http://www.gutenberg.org/ebooks/11?msg=welcome_stranger) in Plain Text UTF-8 format. The downloaded book has been stored in the root directory with the name 'wonderland.txt'. We open this book using the open command and convert all characters into lowercase (so as to reduce the number of characters in the vocabulary, making it easier to learn for the model.)

In [5]:
file = open('wonderland.txt', encoding = 'utf8')
raw_text = file.read()    #you need to read further characters as well
raw_text = raw_text.lower()

Next, we store all the distinct characters occurying in the book in the chars variable. We also remove some of the rare characters (stored in bad-chars) from the book. The final vocabulary of the book is printed at the end of code segment.

In [6]:
chars = sorted(list(set(raw_text)))
print(chars)

bad_chars = ['#', '*', '@', '_', '\ufeff']
for i in range(len(bad_chars)):
    raw_text = raw_text.replace(bad_chars[i],"")

chars = sorted(list(set(raw_text)))
print(chars)

['\n', ' ', '!', '#', '$', '%', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '“', '”', '\ufeff']
['\n', ' ', '!', '$', '%', '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '“', '”']


Next, we summarize the entire book to find that the book consists of a total of 163,721 characters (which is relatively small) and the final number of characters in the vocabulary is 56.

In [7]:
text_length = len(raw_text)
char_length = len(chars)
VOCABULARY = char_length
print("Text length = " + str(text_length))
print("No. of characters = " + str(char_length))

Text length = 163721
No. of characters = 56


Now, we need to create our dataset in the form the model will need. So, we create an input window of 100 characters (SEQ_LENGTH = 100) and shift the window one character at a time until we reach the end of the book. An encoding is used, so as to map each of the characters into it's corresponding location in the vocabulary. Each time the input window contains a new sequence, it is converted into integers, using this encoding and appended to the input list input_strings. For all such input windows, the character just following the sequence is appended to the output list output_strings.

In [8]:
char_to_int = dict((c, i) for i, c in enumerate(chars))
input_strings = []
output_strings = []

for i in range(len(raw_text) - SEQ_LENGTH):
    X_text = raw_text[i: i + SEQ_LENGTH]
    X = [char_to_int[char] for char in X_text]
    input_strings.append(X)    
    Y = raw_text[i + SEQ_LENGTH]
    output_strings.append(char_to_int[Y])

Now, the input_strings and output_strings lists are converted into a numpy array of required dimensions, so that they can be fed to the model for the training.

In [9]:
length = len(input_strings)
input_strings = np.array(input_strings)
input_strings = np.reshape(input_strings, (input_strings.shape[0], input_strings.shape[1], 1))
input_strings = input_strings/float(VOCABULARY)

output_strings = np.array(output_strings)
output_strings = np_utils.to_categorical(output_strings)
print(input_strings.shape)
print(output_strings.shape)

(163621, 100, 1)
(163621, 56)


Now, finally the model is build and then fitted for training. The model is trained for 50 epochs and a batch size of 128 sequences has been used. After every epoch, the current model state is saved if the model has the least loss encountered till that time. We can see that the training loss decreases in almost every epoch till 50 epochs and probably there is further scope to improve the model.

The total training time is ~90 hours on a CPU and ~12 hours if performed on a GPU.

In [10]:
model = buildmodel(VOCABULARY)
filepath="saved_models/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

history = model.fit(input_strings, output_strings, epochs = 50, batch_size = 128, callbacks = callbacks_list)

Epoch 1/50
163621/163621 [==============================] - 6029s - loss: 2.9083  
Epoch 2/50
163621/163621 [==============================] - 5779s - loss: 2.6112  
Epoch 3/50
163621/163621 [==============================] - 5795s - loss: 2.4221  
Epoch 4/50
163621/163621 [==============================] - 5787s - loss: 2.2722  
Epoch 5/50
163621/163621 [==============================] - 5905s - loss: 2.1630  
Epoch 6/50
163621/163621 [==============================] - 5808s - loss: 2.0790  
Epoch 7/50
163621/163621 [==============================] - 5830s - loss: 2.0131  
Epoch 8/50
163621/163621 [==============================] - 5908s - loss: 1.9568  
Epoch 9/50
163621/163621 [==============================] - 5839s - loss: 1.9060  
Epoch 10/50
163621/163621 [==============================] - 5928s - loss: 1.8647  
Epoch 11/50
163621/163621 [==============================] - 6025s - loss: 1.8241  
Epoch 12/50
163621/163621 [==============================] - 7378s - loss: 1.7912  
E

Now that our model has been trained, we can use it for different generating texts as well as predicting next word, which is what we will do in the next notebooks.